# Open search data generator

In [1]:
%cd ..
from opensearchpy import OpenSearch, helpers
import pandas as pd
from pprint import pp
from dotenv import load_dotenv
from tqdm import tqdm
import os
def ppr(resp):
    pp(resp.raw)

c:\ulb\SearchEngines_benchmark\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\ulb\SearchEngines_benchmark


In [2]:
load_dotenv()

# Recupera la variabile della password dall'ambiente
OPENSEARCH_INITIAL_ADMIN_PASSWORD = os.getenv("OPENSEARCH_INITIAL_ADMIN_PASSWORD")
OPENSEARCH_URL = os.getenv("OPENSEARCH_URL")  # Ad esempio http://localhost:9200

# Verifica che la password sia stata caricata correttamente
if OPENSEARCH_INITIAL_ADMIN_PASSWORD:
    print("Password caricata con successo.")
else:
    print("Errore nel caricare la password.")

Password caricata con successo.


In [3]:
print(OPENSEARCH_URL)
print(OPENSEARCH_INITIAL_ADMIN_PASSWORD)

https://localhost:9200
<a123456789A.>


In [4]:
#OPENSEARCH_URL='https://localhost:9200'

In [6]:
client = OpenSearch(
    hosts=[OPENSEARCH_URL],
    http_auth=("admin", OPENSEARCH_INITIAL_ADMIN_PASSWORD),
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)
try:
    info = client.info()
    print("Connessione a OpenSearch avvenuta con successo!")
except Exception as e:
    print(f"Errore nella connessione: {e}")

Connessione a OpenSearch avvenuta con successo!


## Uploading post data

In [18]:
client.indices.delete(index='html_posts', ignore=[400, 404])

{'acknowledged': True}

In [19]:
client.indices.create(
    index='html_posts',
    body = 
    {
    'settings':{
        "analysis": {
            "analyzer": {
                "html_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "char_filter": [
                        "html_strip"
                    ]
                }
            }
        }
    }, 
    'mappings':{
            "properties": {
                "Body": {
                    "type": "text",
                    "analyzer": "html_analyzer"
                },
                "CommentCount": {
                    "type": "integer"
                },
                "CreationDate": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSS"
                },
                "OwnerUserId": {
                    "type": "integer"
                }
            }
        }
    }
)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'html_posts'}

In [20]:
def generator(json_chunk):
    for line in json_chunk:
        yield {
            "_index": "html_posts",
            "_id": line.get('Id'),
            "_source": {
                "Body": line.get("Body",""),
                "CommentCount": line.get("CommentCount",""),
                "CreationDate": line.get("CreationDate",""),
                "OwnerUserId": line.get("OwnerUserId","")
            }
        }


def from_chunk_to_client(chunk):
    json_chunk = chunk.to_dict("records")
    return generator(json_chunk)

In [22]:
columns = ['Id', 'OwnerUserId','Body', 'CommentCount', "CreationDate"]
for chunk in tqdm(pd.read_csv('data/Posts.csv', chunksize=1000)):
    gen = from_chunk_to_client(chunk[columns])
    res = helpers.bulk(client, gen)

0it [00:00, ?it/s]

1566it [12:16,  2.13it/s]


## Trying some quieries

In [23]:
resp = client.search(
    index='html_posts',
    body={ 
        "query": {
            "match": {
                "Body": "I want to"  
            }
        }
    }
)

pp(resp)

{'took': 2548,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
          'max_score': 5.2720766,
          'hits': [{'_index': 'html_posts',
                    '_id': '1859507',
                    '_score': 5.2720766,
                    '_source': {'Body': '<p>I want to display a number in a '
                                        'report, however I only want to show '
                                        'any decimal points if they are '
                                        'present and the I only want to show 1 '
                                        'decimal space.</p>\n'
                                        '\n'
                                        '<p>e.g. if the number is 12 then I '
                                        'want to show 12</p>\n'
                                        '\n'
                                        '<p>If the number is 12.1 then I w

In [24]:
resp = client.search(
    index='html_posts',
    body={ 
        "query": {
            "match": {
                "Body": "<p>"  
            }
        }
    }
)
pp(resp)

{'took': 9,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
          'max_score': None,
          'hits': []}}


In [25]:
resp = client.search(
    index='html_posts',
    body={
        "size": 0,  # We're aggregating, not fetching individual documents
        "query": {
            "bool": {
                "should": [
                    {"match_phrase": {"Body": "sql"}},
                    {"match_phrase": {"Body": "python"}},
                    {"match_phrase": {"Body": " R "}},
                    {"match_phrase": {"Body": "java"}},
                    {"match_phrase": {"Body": "javascript"}},
                    {"match_phrase": {"Body": "c++"}},
                    {"match_phrase": {"Body": "ruby"}},
                    {"match_phrase": {"Body": "php"}}
                ]
            }
        },
        "aggs": {
            "Programming_Language": {
                "filters": {
                    "filters": {
                        "SQL": {"match_phrase": {"Body": "sql"}},
                        "Python": {"match_phrase": {"Body": "python"}},
                        "R": {"match_phrase": {"Body": " R "}},
                        "Java": {"match_phrase": {"Body": "java"}},
                        "JavaScript": {"match_phrase": {"Body": "javascript"}},
                        "C++": {"match_phrase": {"Body": "c++"}},
                        "Ruby": {"match_phrase": {"Body": "ruby"}},
                        "PHP": {"match_phrase": {"Body": "php"}}
                    }
                }
            }
        }
    }
)

In [26]:
pp(resp)

{'took': 260,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
          'max_score': None,
          'hits': []},
 'aggregations': {'Programming_Language': {'buckets': {'C++': {'doc_count': 41095},
                                                       'Java': {'doc_count': 15747},
                                                       'JavaScript': {'doc_count': 28595},
                                                       'PHP': {'doc_count': 20003},
                                                       'Python': {'doc_count': 14099},
                                                       'R': {'doc_count': 6677},
                                                       'Ruby': {'doc_count': 5360},
                                                       'SQL': {'doc_count': 12617}}}}}


In [57]:
all_buckets = []
after_key = None  # Pagination key starts as None

while True:
    # Build the query
    body = {
        "size": 0,  # No document hits, just aggregations
        "query": {
            "bool": {
                "should": [
                    {"match": {"Body": "sql"}},
                    {"match": {"Body": "python"}},
                    {"match": {"Body": " R "}},
                    {"match": {"Body": "java"}},
                    {"match": {"Body": "javascript"}},
                    {"match": {"Body": "c++"}},
                    {"match": {"Body": "ruby"}},
                    {"match": {"Body": "php"}}
                ]
            }
        },
        "aggs": {
            "OwnerUserId": {
                "composite": {
                    "sources": [
                        {"OwnerUserId": {"terms": {"field": "OwnerUserId"}}}
                    ],
                    "size": 5000  # Fetch 1000 buckets per page
                },
                "aggs": {
                    "Programming_Language": {
                        "filters": {
                            "filters": {
                                "SQL": {"match": {"Body": "sql"}},
                                "Python": {"match": {"Body": "python"}},
                                "R": {"match": {"Body": " R "}},
                                "Java": {"match": {"Body": "java"}},
                                "JavaScript": {"match": {"Body": "javascript"}},
                                "C++": {"match": {"Body": "c++"}},
                                "Ruby": {"match": {"Body": "ruby"}},
                                "PHP": {"match": {"Body": "php"}},
                                "Other": {
                                    "bool": {
                                        "must_not": [
                                            {"match": {"Body": "sql"}},
                                            {"match": {"Body": "python"}},
                                            {"match": {"Body": " R "}},
                                            {"match": {"Body": "java"}},
                                            {"match": {"Body": "javascript"}},
                                            {"match": {"Body": "c++"}},
                                            {"match": {"Body": "ruby"}},
                                            {"match": {"Body": "php"}}
                                        ]
                                    }
                                }
                            }
                        },
                        "aggs": {
                            "Post_Count": {
                                "value_count": {"field": "_id"}
                            }
                        }
                    }
                }
            }
        }
    }

    # Add the after_key to the composite aggregation if it's set
    if after_key:
        body["aggs"]["OwnerUserId"]["composite"]["after"] = after_key

    # Execute the search
    resp = client.search(index="html_posts", body=body)

    # Extract buckets
    buckets = resp["aggregations"]["OwnerUserId"]["buckets"]
    all_buckets.extend(buckets)

    # Check if there's more data to fetch
    if "after_key" in resp["aggregations"]["OwnerUserId"]:
        after_key = resp["aggregations"]["OwnerUserId"]["after_key"]
    else:
        break

# Prepare data for DataFrame
data = []
for bucket in all_buckets:
    owner_id = bucket["key"]["OwnerUserId"]
    for lang, lang_bucket in bucket["Programming_Language"]["buckets"].items():
        post_count = lang_bucket["Post_Count"]["value"]
        data.append({
            "OwnerUserId": owner_id,
            "Programming_Language": lang,
            "Post_Count": post_count
        })

# Convert to DataFrame
df = pd.DataFrame(data)
print(df)



        OwnerUserId Programming_Language  Post_Count
0                 0                  C++        2545
1                 0                 Java        1140
2                 0           JavaScript        1736
3                 0                Other           0
4                 0                  PHP        1470
...             ...                  ...         ...
301072      8571118                  PHP           0
301073      8571118               Python           0
301074      8571118                    R           0
301075      8571118                 Ruby           0
301076      8571118                  SQL           0

[301077 rows x 3 columns]


In [58]:
df = df[df['Post_Count']>0]

In [59]:
df.OwnerUserId.nunique()

33453

In [61]:
df.sort_values(by='OwnerUserId')

,OwnerUserId,Programming_Language,Post_Count
0,0,C++,2545
1,0,Java,1140
2,0,JavaScript,1736
8,0,SQL,923
4,0,PHP,1470
...,...,...,...
301047,5696608,R,5
301051,5759201,Java,1
301050,5759201,C++,1
301059,6258565,C++,1
